# Read result file 
    - meta data: available results, mesh, time/freq domain
    - results: displacement and stress

1st step : load dpf client/server, generate documentation if needed

In [1]:
from ansys import dpf
dpf.start_local_server()
#doc = dpf.Operator("html_doc")
#doc.inputs.output_path(r'c:/temp/doc.html')
#doc.run()
import os
res_files = os.getcwd() + r'\..\..\res_files'

server started at --address 10.28.1.60 --port 50052


2nd step : create a model

In [10]:
path = res_files + r'\file.rst'
model = dpf.Model(path)
metadata = model.metadata
results = model.results

In [3]:
print(metadata.result_info)

Modal analysis
Unit system: Metric (m, kg, N, s, V, A)
Physics Type: Mecanic
Available results:
     -  displacement                  
     -  element_nodal_forces          
     -  stress                        
     -  strain                        
     -  temperature                   



In [4]:
print(metadata.meshed_region)

Meshed Region
	55531 nodes
	20821 elements
	Unit: m 



In [5]:
print(metadata.time_freq_support)

Time/Frequency Info:
	Number of sets: 24

With complex values
 
 Cumulative   Frequency (Hz)    Loadstep     Substep     Harmonic index  
     1           5.71e+02          1            1              0         
     2           8.53e+02          1            2              0         
     3           1.97e+03          1            3              0         
     4           2.04e+03          1            4              0         
     5           2.47e+03          1            5              0         
     6           3.06e+03          1            6              0         
     7           5.8e+02           2            1              1         
     8           5.8e+02           2            2              1         
     9           9.52e+02          2            3              1         
     10          9.52e+02          2            4              1         
     11          1.78e+03          2            5              1         
     12          1.78e+03          2            

In [6]:
u = model.results.displacement()
vtk = dpf.Operator("vtk_export")

In [7]:
u.inputs.time_scoping.connect([7])

In [8]:

fields = u.outputs.fields_container()
vtk.inputs.file_path(r"C:\temp\tempsecBase.vtk")
vtk.inputs.mesh.connect(metadata.meshed_region)
vtk.inputs.fields1.connect(fields)
vtk.run()


In [12]:
len(fields)
fields.labels
fields.get_label_space(0)

{'time': 7, 'base_sector': 1}

## expand displacements

In [9]:
displacements = model.operator("mapdl::rst::U_cyclic")

In [10]:
fields = displacements.outputs.fields_container()

In [11]:
disp_field = fields[0]

In [12]:
mesh = displacements.outputs.expanded_meshed_region.get_data()

In [14]:
print(mesh)

Meshed Region
	1945584 nodes
	749556 elements
	Unit:  



In [15]:
print(disp_field)

DPF  Field
	Location: Nodal
	Unit: m
	1945584 id(s)
	data size: 5836752
	shape: (1945584, 3)



## expand stresses

In [20]:
stress = model.operator("mapdl::rst::S_cyclic")

In [21]:
stress.inputs.requested_location.connect("Nodal")

In [22]:
fields = stress.outputs.fields_container()

## export to vtk

In [23]:
vtk = dpf.Operator("vtk_export")

In [24]:
vtk.inputs.file_path(r'c:/temp/file.vtk')
vtk.inputs.fields1.connect(displacements.outputs.fields_container)
vtk.inputs.fields2.connect(stress.outputs.fields_container)
vtk.run()